In [1]:
from phe import paillier
import random, scipy.stats
import numpy as np

In [2]:
# Global 参数
# 设定 workers 数量， objects 数量， 稀疏度
K = 6; M = 5; S = 5

In [3]:
# 初始化数据
XM = np.zeros([K, M])
PHI = np.zeros([K, M])
SPL = [1] * S + [0] * (10 - S)
pk, sk = paillier.generate_paillier_keypair()

In [4]:
# 随机生成每个 worker 的标记向量
for k in range(K):
    PHI[k] = [random.choice(SPL) for m in range(M)]
for i,j in zip(np.sum(PHI, axis = 0), np.sum(PHI, axis = 1)):
    assert i >0 and j > 0 # 确保每个 object 至少有一个 worker 提交

In [5]:
# 随机生成每个 worker 的 sensory data
for k in range(K):
    for m in range(M):
        XM[k][m] = PHI[k][m] * round((random.random()) * 10, 2) # 随机在 [0,10) 之间取值

In [6]:
def catd_weight_update(PHI, XM, XT):
    Y = np.array([scipy.stats.chi2.ppf(0.05, k) for k in np.sum(PHI, axis = 1)])
    W = Y / np.sum(PHI * (XM - XT) ** 2, axis = 1)
    return W

In [7]:
def catd_truth_update(PHI, XM, W):
    XT = np.dot(W, XM) / np.dot(W, PHI)
    return XT

In [8]:
# Y = np.array([scipy.stats.chi2.ppf(0.05, k) for k in np.sum(PHI, axis = 1)])

In [9]:
# 分割数据
def secure_share(PHI, XM):
    Y = np.array([scipy.stats.chi2.ppf(0.05, k) for k in np.sum(PHI, axis = 1)])
    tphi = PHI / np.tile(Y, M).reshape(M, K).T
    tphi0 = [[] for k in range(K)]
    phi0 = [[] for k in range(K)]
    xm0 = [[] for k in range(K)]
    for k in range(K):
        tphi0[k] = [ round(random.random(),2) for m in range(M)]
        phi0[k] = [ round(random.random(),2) for m in range(M)]
        xm0[k] = [ round(random.random(),2) for m in range(M)]
    tphi0 = np.array(tphi0)
    phi0 = np.array(phi0)
    xm0 = np.array(xm0)
    tphi1 = tphi - tphi0
    phi1 = PHI - phi0
    xm1 = XM - xm0
    return [(tphi0, phi0, xm0), (tphi1, phi1, xm1)]

In [10]:
[S0_Data, S1_Data] = secure_share(PHI, XM)

In [11]:
def weight_update_s0(tphi0, phi0, xm0):
    M0 = np.sum(tphi0 * (xm0 ** 2), axis = 1) 
    C0 = []
    for m in M0:
        C0.append(pk.encrypt(m))
    c_xm0 = np.array([pk.encrypt(m) for m in xm0.reshape(K*M,)]).reshape(K, M)
    c_xm02 = np.array([pk.encrypt(m) for m in (xm0 ** 2).reshape(K*M,)]).reshape(K, M)
    c_tphi0 = np.array([pk.encrypt(m) for m in tphi0.reshape(K*M,)]).reshape(K, M)
    c_phi0 = np.array([pk.encrypt(m) for m in phi0.reshape(K*M,)]).reshape(K, M)
    c_tphi0_xm0 = np.array([pk.encrypt(m) for m in (tphi0 * xm0).reshape(K*M,)]).reshape(K, M)
    C_pack1 = (c_xm0, c_xm02, c_tphi0, c_phi0, c_tphi0_xm0)
    return [C0, C_pack1]

In [12]:
# (tphi1, phi1, xm1) = S1_Data
[C0, C_pack1] = weight_update_s0(*S0_Data)

In [13]:
# def weight_update_s1(thpi1, phi1, xm1, XT, C0, C_pack1):
#     tphi1 * C_pack1[1]

In [14]:
XT = np.sum(XM, axis = 0) / np.sum(PHI, axis = 0)
# XT = np.array([round((random.random())*10, 2) for m in range(M)])

In [15]:
tw = catd_weight_update(PHI, XM, XT)
tw

array([0.01179855, 0.02388402, 0.02534397, 0.02646769, 0.00841197,
       0.00606093])

In [16]:
def weight_update_s1(tphi1, phi1, xm1, C0, C_pack1, XT):
    C1 = np.sum(tphi1 * C_pack1[1], axis = 1)
    C2 = np.sum(np.multiply(2, (xm1 - XT)) * (C_pack1[4] + tphi1 * C_pack1[0]), axis = 1)
    C3 = np.sum((xm1 - XT) ** 2 * C_pack1[2], axis = 1)
    C4 = np.array([pk.encrypt(m) for m in np.sum(tphi1 * (xm1 - XT) ** 2, axis = 1)])
    return 1 / np.array([sk.decrypt(c) for c in C0 + C1 + C2 + C3 + C4])

In [17]:
weight_update_s1(*S1_Data, C0, C_pack1, XT)

array([0.01179855, 0.02388402, 0.02534397, 0.02646769, 0.00841197,
       0.00606093])

In [18]:
def truth_update_s0(tw, tphi0, phi0, xm0):
    c_tw = np.array([pk.encrypt(m) for m in tw])
    c_tw_xm0 = np.array([pk.encrypt(m) for m in (np.tile(tw, M).reshape(M, K).T * xm0).reshape(K*M,)]).reshape(K, M)
    c_tw_phi0 = np.array([pk.encrypt(m) for m in (np.tile(tw, M).reshape(M, K).T * phi0).reshape(K*M,)]).reshape(K, M)
    return (c_tw, c_tw_xm0, c_tw_phi0)

In [19]:
C_pack2 = truth_update_s0(tw, *S0_Data)

In [20]:
def truth_update_s1(tphi1, phi1, xm1, C_pack2):
    C5 = np.sum(C_pack2[1] + np.tile(C_pack2[0], M).reshape(M, K).T * xm1, axis = 0)
    C6 = np.sum(C_pack2[2] + np.tile(C_pack2[0], M).reshape(M, K).T * phi1, axis = 0)
    return [C5, C6]

In [21]:
[C5, C6] = truth_update_s1(*S1_Data, C_pack2)

In [22]:
[sk.decrypt(wx) / sk.decrypt(wphi) for wx,wphi in zip(C5, C6)]

[3.572460879154833,
 6.869581545807172,
 4.73580466064769,
 5.5859344005910065,
 6.170921583966461]

In [23]:
catd_truth_update(PHI, XM, tw)

array([3.57246088, 6.86958155, 4.73580466, 5.5859344 , 6.17092158])